In [7]:
import numpy as np
import math
from scipy.stats import norm

## Asian options pricing

In [8]:
def pricer_mm(S0, K, r, sigma, T, n, option_type):
    
#     M1 = S0 /(n+1)*sum([np.exp(r*i/n) for i in range(n+1)])
#     M2 = S0**2/(n+1)**2 * sum([np.exp(r*(i+j)/n + (sigma**2)*min(i,j)/n) for i in range(n+1) for j in range(n+1)])
    M1 = S0 /n*sum([np.exp(r*(i+1)/n) for i in range(n)])
    M2 = S0**2/n**2 * sum([np.exp(r*(i+j+2)/n + (sigma**2)*min(i+1,j+1)/n) for i in range(n) for j in range(n)])
    param1 = 2*np.log(M1) - np.log(M2)/2
    param2 = math.sqrt(np.log(M2) - 2*np.log(M1))
    d1 = (np.log(1/K) + param1 + param2**2) /param2
    d2 = d1 - param2
    if option_type == 'call':
        price = np.exp(-r*T) * (np.exp(param1+(param2**2)/2)*norm.cdf(d1) - K*norm.cdf(d2) )
    elif option_type == 'put':
        price = np.exp(-r*T) * (K*norm.cdf(-d2) - (np.exp(param1 +( param2**2)/2) ) * norm.cdf(-d1) )
    else:
        raise ValueError("Option type must be either 'call' or 'put'.")

    return price

In [9]:
def pricer_mc(S0, K, r, sigma, T, n, M, option_type):
    
    dt = T/n
    ranvec = np.random.normal(0, 1,(M, n))

    Spath1 = S0*np.concatenate((np.ones((M,1)),np.cumprod(np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(ranvec)), axis=1)), axis=1)
    Spath2 = S0*np.concatenate((np.ones((M,1)),np.cumprod(np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(-ranvec)), axis=1)), axis=1)
    U1 = np.average(Spath1, axis=1)
    U2 = np.average(Spath2, axis=1)
    C = 0.5*np.exp(-r*T)*(np.maximum(U1 - K, 0) + np.maximum(U2 - K, 0) )
    P = 0.5*np.exp(-r*T)*(np.maximum(K - U1, 0) + np.maximum(K - U2, 0) )
    price_c = np.mean(C)
    price_p = np.mean(P)
    stderr_c = np.std(C)/np.sqrt(M)
    stderr_p = np.std(P)/np.sqrt(M)
    confint_c = [price_c - 1.96*stderr_c, price_c + 1.96*stderr_c]
    confint_p = [price_p - 1.96*stderr_p, price_p + 1.96*stderr_p]
    if option_type == 'call':
        price = price_c
        confint = confint_c
    elif option_type == 'put':
        price = price_p
        confint = confint_p
    else:
        raise ValueError("Option type must be either 'call' or 'put'.")

    return [price] + confint

In [94]:
S0 = 130.17
K = 100
r = 0.05
sigma = 0.484
T = 1
n = 100
option_type = "put"
M = 2


In [95]:
M1 = S0 /n*sum([np.exp(r*(i+1)/n) for i in range(n)])
print(M1)
M2 = S0**2/n**2 * sum([np.exp(r*(i+j+2)/n + (sigma**2)*min(i+1,j+1)/n) for i in range(n) for j in range(n)])
print(M2)
param1 = 2*np.log(M1) - np.log(M2)/2
print(param1)
param2 = math.sqrt(np.log(M2) - 2*np.log(M1))
print(param2)
d1 = (np.log(1/K) + param1 + param2**2) /param2
print(d1)
d2 = d1 - param2
print(d2)
print(np.exp(-r*T))
price = np.exp(-r*T) * (np.exp(param1+(param2**2)/2)*norm.cdf(d1) - K*norm.cdf(d2) )
print(price)

133.51254487923345
19344.921981767002
4.853298268049465
0.28599711365275
1.1535865759808366
0.8675894623280866
0.951229424500714
34.42807314132599


In [92]:
 (np.exp(param1+(param2**2)/2)*norm.cdf(d1) - K*norm.cdf(d2) )

36.193238197395715

In [93]:
np.exp(-r*T)

0.951229424500714

In [96]:
pricer_mm(S0, K, r, sigma, T, n, option_type)

2.5499543622983545

In [65]:
pricer_mc(S0, K, r, sigma, T, n, M, option_type)

[2.748542653690688, 1.5049059202853021, 3.9921793870960736]

In [ ]:
np.concatenate((np.ones((M,1)),np.cumprod(np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(ranvec)), axis=1)), axis=1)

In [70]:
np.random.normal(0, 1,(M, n))

array([[ 0.48365333, -0.63718807,  0.19488429,  1.53101143,  0.64164264],
       [ 0.58478757,  0.8351021 , -0.637963  , -0.75709799, -1.03089092]])

In [75]:
np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(ranvec))

array([[1.345644  , 1.1046238 , 1.03630287, 1.08173319, 1.07764883],
       [0.99188214, 1.02475468, 1.09520924, 1.04497134, 1.04226809]])

In [76]:
np.cumprod(np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(ranvec)), axis=1)

array([[1.345644  , 1.48643039, 1.54039208, 1.66629324, 1.79567896],
       [0.99188214, 1.01643587, 1.11320995, 1.16327249, 1.2124418 ]])

## Barrier options pricing

In [31]:
'''
Vanilla Call option price
Inputs:
 k :strike
 r: interest rate
 x: underlying price at time 0
 tau: time to expiry
 sigma: volatility
    
'''
def price_european_call(K, r, S0, tau, sigma):
    d1 = (np.log(S0/K) + (r+0.5*(sigma**2))*tau) / (sigma*np.sqrt(tau))
    d2 = (np.log(S0/K) + (r-0.5*(sigma**2))*tau) / (sigma*np.sqrt(tau))
    return S0*norm.cdf(d1)-K*np.exp(-r*tau)*norm.cdf(d2)

'''
Down-and-In Single Lower Barrier Call option price from call option
Additional inputs:
  B: (lower) barrier level at t=0
'''
def price_barrier_down_in_call(K, r, S0, tau, sigma, B):
    misc = r-(sigma**2)/2
    return (B/S0)**(2*misc/(sigma**2))*call_price(K,r,(B**2)/S0,tau,sigma)

'''
Down-and-Out Single Lower Barrier Call option price from call option
'''
def price_barrier_down_out_call(K, r, S0, tau, sigma, B):
    return call_price(K, r, S0, tau, sigma) - price_barrier_down_in_call(K, r, S0, tau, sigma, B)

'''
Simulate Geometric Brownian motion several times
Inputs:
  S0: underlying price at time 0
  r: return
  sigma: volatility
  T: time to expiry
  n: path length (number of monitoring points)
  M: number of paths
Output: matrix, rows contain the MC paths 
'''
def simulate_GBM(S0, r, sigma, T, n, M):
    dt = T/n
    ranvec = np.random.normal(0, 1,(M, n))
    return S0*np.concatenate((np.ones((M,1)),np.cumprod(np.exp((r-0.5*sigma**2)*dt + sigma*np.sqrt(dt)*(ranvec)), axis=1)), axis=1)

'''
Down-and-out call barrier pricer with normal Monte Carlo
'''
def price_barrier_down_in_mc(K, r, S0, sigma, B, T, n, M):
  simi = simulate_GBM(S0, r, sigma, T, n, M)
  pat=c(1:m)
  rem=sapply(pat, function(i){
    ok=((min(sim[ ,i])>L) & (sim[nrow(sim),i]>k))
    return(ok) })
  sum(sim[nrow(sim),rem]-k)/m*exp(-r*tt)
}


In [32]:
K = 100
r = 0.05
S0 = 130.17
T = 1
tau = 1
sigma = 0.484
B = 90
n = 5
M = 2

In [36]:
print(price_european_call(K, r, S0, tau, sigma))
print(price_barrier_down_in_call(K, r, S0, tau, sigma, B))
print(price_barrier_down_out_call(K, r, S0, tau, sigma, B))
print(simulate_GBM(S0, r, sigma, T, n, M))

43.30739589609889
4.7350438619284505
38.572352034170436
[[130.17       136.63216068 136.60421398 135.77757379 167.94099427
  123.59885799]
 [130.17       141.43500925 128.25667684 138.04207452 163.88793101
  162.63008917]]


In [50]:
simi = simulate_GBM(S0, r, sigma, T, n, M)
print(simi)
min(simulate_GBM(S0, r, sigma, T, n, M))

[[130.17       133.45386217 217.61880543 175.8485371  146.96686435
  122.31279252]
 [130.17       126.85713176 154.32630077 144.04788805 102.31047629
  105.34688541]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [61]:
np.min(simi, 1)

array([122.31279252, 102.31047629])